In [3]:
import numpy as np 

# Suppress scientific notation
np.set_printoptions(suppress=True)

# Generate randomly distributed parameters
params = np.random.uniform(low=-50, high=150, size=20)

# mAKE sure important values are at the beginning for better debugging
params[0] = params.max() + 1
params[1] = params.min() - 1
params[2] = 0

# Round each number to the second decimal place
params = np.round(params, 2)

print(params)

[143.56 -45.88   0.   -18.63  26.27  38.18 -22.46  27.82 -44.88 -13.47
  -7.   -26.85  -7.49 131.85 131.71  -3.08  74.83 142.56  90.14  86.44]


### Define quantiation methods and quantize

In [8]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q

def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    # Calculate the scale and zero point
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits-1)
    zero = -1*np.round(beta / scale)
    lower_bound, upper_bound = 0, 2**bits-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantize(params_q: np.array, scale: float, zero: int) -> np.array:
    return (params_q - zero) * scale 

def symmetric_dequantize(params_q: np.array, scale: float) -> np.array:
    return params_q * scale

def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    # Calculate the scale
    alpha = np.max(np.abs(params))
    scale = alpha / (2**(bits-1)-1)
    lower_bound = -2**(bits-1)
    upper_bound = 2**(bits-1)-1
    # Quantize the parameters
    quantized = clamp(np.round(params / scale), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale

def quantization_error(params: np.array, params_q: np.array):
    # calculate the MSE
    return np.mean((params - params_q)**2)

(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original:
[143.56 -45.88   0.   -18.63  26.27  38.18 -22.46  27.82 -44.88 -13.47
  -7.   -26.85  -7.49 131.85 131.71  -3.08  74.83 142.56  90.14  86.44]

Asymmetric scale: 0.7429019607843137, zero: 62.0
[255   0  62  37  97 113  32  99   2  44  53  26  52 239 239  58 163 254
 183 178]

Symmetric scale: 1.1303937007874016
[127 -41   0 -16  23  34 -20  25 -40 -12  -6 -24  -7 117 117  -3  66 126
  80  76]


In [9]:

# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(params, 2))
print('')
print(f'Dequantize Asymmetric:')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[143.56 -45.88   0.   -18.63  26.27  38.18 -22.46  27.82 -44.88 -13.47
  -7.   -26.85  -7.49 131.85 131.71  -3.08  74.83 142.56  90.14  86.44]

Dequantize Asymmetric:
[143.38 -46.06   0.   -18.57  26.    37.89 -22.29  27.49 -44.57 -13.37
  -6.69 -26.74  -7.43 131.49 131.49  -2.97  75.03 142.64  89.89  86.18]

Dequantize Symmetric:
[143.56 -46.35   0.   -18.09  26.    38.43 -22.61  28.26 -45.22 -13.56
  -6.78 -27.13  -7.91 132.26 132.26  -3.39  74.61 142.43  90.43  85.91]


In [10]:
# Calculate the quantization error
print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(params, params_deq_asymmetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(params, params_deq_symmetric), 2)}')

  Asymmetric error: 0.05
   Symmetric error: 0.11
